In [ ]:
import numpy as np 
import xarray as xr 


In [ ]:
full_size = True
if full_size:
    SIDE = 131
    MIDDLE = 65
else:
    SIDE = 11
    MIDDLE = 5

In [ ]:
arr = np.zeros((SIDE, SIDE), dtype=np.bool_)
assert SIDE // 2 == MIDDLE
starting_pos  = MIDDLE
arr[MIDDLE][MIDDLE] # starting pos

In [ ]:
xda = xr.DataArray(
        arr,
        coords={
            "row": list(range(arr.shape[0])),
            "col": list(range(arr.shape[1])),
        },)
xda


In [ ]:
def create_diamond_mask(xda) -> xr.DataArray:
    diamond_arr = np.abs((xda.row.values[:, None] - (xda.row.size // 2))) + (
        np.abs(xda.col.values - (xda.row.size // 2))
    )

    xda = xda.copy(data=diamond_arr)
    assert xda.sel(row=xda.row.size // 2, col=xda.row.size // 2) == 0
    diamond_mask = (xda.where(np.abs(xda) <= xda.row.size // 2, -1) >= 0).astype(
        np.uint8
    )
    return diamond_mask

In [ ]:
diamond_arr = np.abs((xda.row.values[:, None] - (xda.row.size // 2))) + (
    np.abs(xda.col.values - (xda.row.size // 2))
)
xda = xda.copy(data=diamond_arr)
assert xda.sel(row=xda.row.size // 2, col=xda.row.size // 2) == 0
xda

In [ ]:
xda[MIDDLE]

In [ ]:
diamond_mask = (xda.where(np.abs(xda) <= xda.row.size // 2, -1) >= 0).astype(np.uint8)

In [ ]:
top_left = diamond_mask[:MIDDLE, :MIDDLE]
top_left

In [ ]:
top_left = diamond_mask[:MIDDLE+1, :MIDDLE+1]
top_left

## With input

In [ ]:
from pathlib import Path
from advent_of_code.common import get_input_file_path
from advent_of_code.year_2023.year_2023_day_21 import (
    parse_text_input,
)


text = (Path("../") / get_input_file_path(2023, 21)).read_text()
garden = parse_text_input(text)
garden

In [ ]:
free = ((garden == b"." )| (garden == b"S"))

In [ ]:
free.plot()

In [ ]:
(~free).plot()

In [51]:
create_diamond_mask(free).plot()

Error in callback <function _draw_all_if_interactive at 0x7f6e7fcc5ea0> (for post_execute), with arguments args (),kwargs {}:


### Observation on Data

- All edges are free
- The middle row and middle cols are free
- Presence of a diagonal path

Hence the image can be safely divided into 8 half-quadrants

In [ ]:
free

In [ ]:
assert (free[0]).all().item()
assert (free[-1]).all().item()
assert (free[:, 0]).all().item()
assert (free[:, -1]).all().item()
assert free[MIDDLE].all().item()
assert free[:, MIDDLE].all().item()

In [ ]:
from advent_of_code.year_2023.year_2023_day_21 import get_starting_position, run_steps_details
initial_pos = get_starting_position(garden)
max_iter = 65 * 2
# max_iter = 6

history, reached, reached_even_xda, reached_odd_xda= run_steps_details(garden, initial_pos, max_iter)
...


In [ ]:
import matplotlib.pyplot as plt
reached_odd_xda.plot.imshow()

In [ ]:
reached_even_xda.plot.imshow()

### Check part 1 can be found again

In [ ]:
max_iter = 64 # we will want the even
_, _, reached_even_xda_p1, reached_odd_xda_p1 = run_steps_details(
    garden, initial_pos, max_iter
)

In [ ]:
diamond_1x1 = create_diamond_mask(reached_even_xda_p1)
diamond_1x1.plot.imshow()

In [ ]:
assert reached_even_xda_p1.where(diamond_1x1, 0).sum() == 3740

### 1x1

In [ ]:
odd = reached_odd_xda
eve = reached_even_xda

In [ ]:
odd.where(create_diamond_mask(odd), 0).sum().item()

In [ ]:
assert eve.where(create_diamond_mask(eve), 0).sum().item() == 3740 
# Part 1 also works when using the eve diagram (as 65 = 64 + 1, we juste iterated one more time)

### 3x3 concat

!!!

Note: The step count is EVEN, so even must be placed back at the center of the mosaic


In [ ]:
assert MIDDLE + 1 * odd.row.size == 196  # 65 + 1 * 131

In [ ]:
new_coords3 = list(range(3 * odd.row.size))
concat3 = xr.concat(
    [
        xr.concat([eve, odd, eve], dim="col"),
        xr.concat([odd, eve, odd], dim="col"),
        xr.concat([eve, odd, eve], dim="col"),
    ],
    dim="row",
).assign_coords(dict(row=new_coords3, col=new_coords3))
concat3

In [ ]:
concat3.plot.imshow(size=30)

In [ ]:
diamond_3x3 = create_diamond_mask(concat3)
diamond_3x3.plot.imshow()

In [ ]:
concat3.where(diamond_3x3, 0).sum().item()

In [ ]:
concat3

### 5x5 concat

Note: The step count is odd, so odd must be placed back at the center of the mosaic

In [ ]:
assert MIDDLE + 2 * odd.row.size == 327 # 65 + 2 * 131

In [ ]:
new_coords5 = list(range(5 * odd.row.size))
concat5 = xr.concat(
    [
        xr.concat([odd, eve, odd, eve, odd], dim="col"),
        xr.concat([eve, odd, eve, odd, eve], dim="col"),
        xr.concat([odd, eve, odd, eve, odd], dim="col"),
        xr.concat([eve, odd, eve, odd, eve], dim="col"),
        xr.concat([odd, eve, odd, eve, odd], dim="col"),
    ],
    dim="row",
).assign_coords(dict(row=new_coords5, col=new_coords5))
concat5

In [ ]:
concat5.plot.imshow()

In [ ]:
diamond_5x5 = create_diamond_mask(concat5)
diamond_5x5.plot.imshow()

In [ ]:
concat5.where(diamond_5x5, 0).sum().item()

In [ ]:
# Thanks https://www.reddit.com/r/adventofcode/comments/18nevo3/comment/kebm6ak/?utm_source=share&utm_medium=web3x&utm_name=web3xcss&utm_term=1&utm_content=share_button
# Not having to lagrange myself
# The area growing is x**2, so 3 points are required to interpolate, hence the previous computing
# of 1x1, 3x3 and 5x5 grids
def evaluate_quadratic_equation(x, y, target):
    # Fit a quadratic polynomial (degree=2) through the points
    coefficients = np.polyfit(x, y, 2)

    # Evaluate the quadratic equation at the given target value
    result = np.polyval(coefficients, target)

    return round(result)

In [ ]:
import numpy as np

x = (0, 1, 2)
y = (
    odd.where(create_diamond_mask(odd), 0).sum().item(),
    concat3.where(diamond_3x3, 0).sum().item(),
    concat5.where(diamond_5x5, 0).sum().item())


In [ ]:
evaluate_quadratic_equation(x, y, 0) == y[0]

In [ ]:
evaluate_quadratic_equation(x, y, 1) == y[1]

In [ ]:
evaluate_quadratic_equation(x, y, 2) == y[2]

In [ ]:
result = evaluate_quadratic_equation(x, y, 26501365 // 131)

In [ ]:
635572702833258 # Too high
# The error was because I forgot to put the even array at the center of the 3x3 mosaic
# Each macro step changes the parity

In [ ]:
assert result < 635572702833258

In [ ]:
result